In [22]:
from youtube_transcript_api import YouTubeTranscriptApi
from IPython.display import display, Markdown
import re, os
from groq import Groq
from dotenv import load_dotenv

In [23]:
def getVideoTranscript(video_id):
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(video_id)

    transcriptions = ""
    for text in transcript.snippets:
        transcriptions += f"{text.text} "
    
    return transcriptions

def splitVideoUrl(url):
    videoId = re.split("=", url)
    return videoId[1]

In [24]:
load_dotenv(override=True)
api_key = os.getenv('GROQ_API_KEY')

In [25]:
def generateSummary(url):    
    client = Groq(api_key=api_key)
    videoTranscript = getVideoTranscript(splitVideoUrl(url))
    messages = [
    {
        "role": "system",
        "content": "You are an expert video content summarizer. Always produce clear, concise, and accurate summaries strictly in markdown."
    },
    {
        "role": "user",
        "content": f"Summarize the following video transcript.\n\nGoals:\n- Focus on core ideas, key arguments, and main takeaways\n- Remove filler, repetition, and irrelevant details\n- Do NOT add opinions or external information\n- Preserve important facts, names, and conclusions\n\n---\n\n### Output Options (select one or more):\n1. **Ultra-Short Summary**: 2–3 bullet points, ≤60 words\n2. **Concise Summary (Default)**: 5–7 bullet points, ~120–150 words\n3. **Detailed Summary**: Structured paragraphs or sections\n4. **Key Takeaways**: Actionable or memorable insights only\n5. **Step-by-Step Breakdown**: Numbered steps (for tutorials/technical videos)\n6. **Topic-Based Summary**: Group points under clear headings\n\n---\n\n### Context-Aware Emphasis:\n- Educational → concepts and learning outcomes\n- Technical → tools, steps, methods, results\n- Talk / Interview → viewpoints and insights\n- News / Commentary → claims, evidence, conclusions\n\n---\n\n### Transcript:\n{videoTranscript}"
    }
    ]

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages
    )
    return response.choices[0].message.content

In [26]:
def displaySummary(url):
    summary = generateSummary(url)
    display(Markdown(summary))

In [27]:
if __name__ == "__main__":
    url = input("Enter YouTube URL: ")
    displaySummary(url)


### Concise Summary
* The speaker shares 8 Notion use cases from their daily workflow, including creating a "notes thread" for recurring meetings.
* They recommend including an "outstanding" section at the top of every notes thread page to track unresolved issues.
* The speaker uses the year-quarter-topic naming convention for notes thread pages, allowing for easy filtering and searching.
* Notion AI is used to generate new content, such as monthly recap emails, based on previous templates and context.
* Creating an "inbox view" for databases helps catch items that don't have a clear home, such as tasks without due dates.
* The speaker distinguishes between the "raw database" and "views" of that database, using filters to create actionable workspaces.
* Buttons are used to simplify tasks, such as updating project status or capturing new prompts.
* Synced blocks are used to reflect changes across all instances, reducing errors and saving time.